In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch
import optuna
import os
import pytorch_lightning as pl
import yaml

from Data.Drosophilla.FlyDataMod import FlyDataModule
from IPython.core.debugger import set_trace
from Models import LinearRegression as lrg
from torch import nn as nn
from Utils import callbacks as cb
from Utils import evaluations as ev
from Utils import HyperParams as hp
from Utils import loggers as lg

np.random.seed(0)

In [ ]:
def runLR_Exp(
    trial,
    lr,
    batch_size,
    label_type,
    label_val,
    hasRidge,
    hasLasso,
    root_dir,
    ):
    logger  = lg.DictLogger(trial.number,
                           root_dir)
    trainer = pl.Trainer(
        gpus=1,
        logger=logger,
        max_epochs=50,
        callbacks=[cb.getcb()],
        default_root_dir=root_dir)
    
    #"Experiments/Table_4_Linear_Regression_HP_Tuning")

    dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=batch_size,
                  label_type=label_type,
                  label_val=label_val)
    dm.setup()

    hparams={'cell_line':"S2",
            "data_win_radius":5,
            "label_type":label_type,
            "batch_size":batch_size}
    model_linear  = lrg.LinearRegressionModule(
        inputSize=29,
        outputSize=1,
        hasRidge=hasRidge,
        hasLasso=hasLasso,
        lr=lr,
        hparams=hparams)
    model_linear.cuda()

    trainer.fit(model_linear, dm)
    score = logger.metrics[-1]['val weighted mse loss']
    if np.isnan(score):
        score = 9999999
    return score

def objective(trial):
    lr = trial.suggest_categorical("lr", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    batch_size = trial.suggest_categorical("batch_size", [1, 4, 16, 64])
    return runLR_Exp(
        trial=trial,
        lr=lr,
        batch_size=batch_size,
        label_type=label_type,
        label_val=label_val,
        hasRidge=hasRidge,
        hasLasso=hasLasso,
        root_dir=root_dir) 

In [ ]:
#Table 4 linear regress lambda
label_type  = "gamma"
label_val = 0
for hasLasso, hasRidge in [(False, False),
                           (True, True),
                           (True, False),
                           (False, True)]:
    if not os.path.isdir("Experiments/Table_4_Linear_Regression_Tunning"):
        os.mkdir("Experiments/Table_4_Linear_Regression_Tunning")
    root_dir = "Experiments/Table_4_Linear_Regression_Tunning/"+label_type+"_"+str(label_val)+"_R"+str(hasRidge)+"_L"+str(hasLasso)
    if not os.path.isdir(root_dir):
        os.mkdir(root_dir)
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)
    hp.save_hyperparams(root_dir,
                    study.best_trial.number)

    

In [ ]:
#Table 5 linear regress lambda
label_type  = "insulation"
label_val = 3
for hasLasso, hasRidge in [(False, False),
                           (True, True),
                           (True, False),
                           (False, True)]:
    root_dir = "Experiments/Table_5_Linear_Regression_Tunning/"+label_type+"_"+str(label_val)+"_R"+str(hasRidge)+"_L"+str(hasLasso)
    if not os.path.isdir("Experiments/Table_5_Linear_Regression_Tunning/"):
        os.mkdir("Experiments/Table_5_Linear_Regression_Tunning/")
    if not os.path.isdir(root_dir):
        os.mkdir(root_dir)
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)
    hp.save_hyperparams(root_dir,
                    study.best_trial.number)

In [ ]:
#Table 6 linear regress lambda
label_type  = "directionality"
label_val = 10
for hasLasso, hasRidge in [(False, False),
                           (True, True),
                           (True, False),
                           (False, True)]:
    root_dir = "Experiments/Table_6_Linear_Regression_Tunning/"+label_type+"_"+str(label_val)+"_R"+str(hasRidge)+"_L"+str(hasLasso)
    if not os.path.isdir("Experiments/Table_6_Linear_Regression_Tunning/"):
        os.mkdir("Experiments/Table_6_Linear_Regression_Tunning/")
    if not os.path.isdir(root_dir):
        os.mkdir(root_dir)
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)
    hp.save_hyperparams(root_dir,
                    study.best_trial.number)

In [ ]:
dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type='insulation',
                  label_val=10)
dm.setup()
layer_weights = glob.glob("Experiments/Table_5_Linear_Regression_Tunning/insulation_10_RFalse_LFalse/optuna/version_0/checkpoints/*")[0]
model         = lrg.LinearRegressionModule.load_from_checkpoint(layer_weights).to("cuda:0")
ev.createPlot(model, dm, "train", "idk")

In [ ]:
#display Table 4
label_type  = "gamma"
label_val = 0
hasRidge   = False
hasLasso   = False



vals = []
for hadRidge, hasLasso in [(True, True),
                          (True, False),
                          (False, True),
                          (False, False)]:
    root_dir = "Experiments/Table_4_Linear_Regression_Tunning/"+label_type+"_"+str(label_val)+"_R"+str(hasRidge)+"_L"+str(hasLasso)
    dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
    dm.setup()
    exps = sorted(glob.glob(root_dir+"/optuna/*"))
    for e, exp in enumerate(exps):
        layer_weights = glob.glob(exp+"/checkpoints/*")[0]
        hparams       = yaml.full_load(open(glob.glob(exp+"/hparams.yaml")[0],'r'))
        model = lrg.LinearRegressionModule.load_from_checkpoint(layer_weights).to("cuda:0")
        row = []
        row.append(hparams['lr'])
        row.append(hparams['hparams']['batch_size'])
        mm  = ev.getModelMetrics(model, dm, 'val').values()
        fmm = list(map(lambda x: "{:.2f}".format(x), mm))
        row.extend(fmm)
        vals.append(row)
        

        
cols=['lr','b_sz','val_mse','val_mae','val_r2','val_pcc','val_spc']
fig, ax = plt.subplots(1, figsize=(20,20))
table   = ax.table(vals,
                cellLoc="center",
                colLabels=cols,
                rowLabels=list(range(0,len(vals))))

table.set_fontsize(14)
ax.axis('off')
plt.show()

#to csv
# TODO problem, right now the results are split by regression types
import pandas as pd
ar_val   = np.array(vals)
ar_col   = np.expand_dims(np.array(cols),0)
csv_data = np.vstack((ar_col, ar_val))
pd.DataFrame(csv_data).to_csv("Experiments/Table_4_Linear_Regression_Tunning/Sup_4_Results.csv")

In [ ]:
#display Table 5
label_type  = "insulation"
label_val = 3
hasRidge   = False
hasLasso   = False

vals = []
for hadRidge, hasLasso in [(True, True),
                          (True, False),
                          (False, True),
                          (False, False)]:
    root_dir = "Experiments/Table_5_Linear_Regression_Tunning/"+label_type+"_"+str(label_val)+"_R"+str(hasRidge)+"_L"+str(hasLasso)
    print(root_dir)
    dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
    dm.setup()
    exps = sorted(glob.glob(root_dir+"/optuna/*"))
    for e, exp in enumerate(exps):
        print(e)
        layer_weights = glob.glob(exp+"/checkpoints/*")[0]
        hparams       = yaml.full_load(open(glob.glob(exp+"/hparams.yaml")[0],'r'))
        model = lrg.LinearRegressionModule.load_from_checkpoint(layer_weights).to("cuda:0")
        row = []
        row.append(hparams['lr'])
        row.append(hparams['hparams']['batch_size'])
        mm  = ev.getModelMetrics(model, dm, 'val').values()
        fmm = list(map(lambda x: "{:.2f}".format(x), mm))
        row.extend(fmm)
        vals.append(row)
        

        
cols=['lr','b_sz','val_mse','val_mae','val_r2','val_pcc','val_spc']
fig, ax = plt.subplots(1, figsize=(20,20))
table   = ax.table(vals,
                cellLoc="center",
                colLabels=cols,
                rowLabels=list(range(0,len(vals))))

table.set_fontsize(14)
ax.axis('off')
plt.show()

#to csv
# TODO problem, right now the results are split by regression types
import pandas as pd
ar_val   = np.array(vals)
ar_col   = np.expand_dims(np.array(cols),0)
csv_data = np.vstack((ar_col, ar_val))
pd.DataFrame(csv_data).to_csv("Experiments/Table_5_Linear_Regression_Tunning/Sup_5_Results.csv")

In [ ]:
#display Table 6
label_type  = "directionality"
label_val = 10
hasRidge   = False
hasLasso   = False



vals = []
for hadRidge, hasLasso in [(True, True),
                          (True, False),
                          (False, True),
                          (False, False)]:
    root_dir = "Experiments/Table_6_Linear_Regression_Tunning/"+label_type+"_"+str(label_val)+"_R"+str(hasRidge)+"_L"+str(hasLasso)
    dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
    dm.setup()
    exps = sorted(glob.glob(root_dir+"/optuna/*"))
    for e, exp in enumerate(exps):
        layer_weights = glob.glob(exp+"/checkpoints/*")[0]
        hparams       = yaml.full_load(open(glob.glob(exp+"/hparams.yaml")[0],'r'))
        model = lrg.LinearRegressionModule.load_from_checkpoint(layer_weights).to("cuda:0")
        row = []
        row.append(hparams['lr'])
        row.append(hparams['hparams']['batch_size'])
        mm  = ev.getModelMetrics(model, dm, 'val').values()
        fmm = list(map(lambda x: "{:.2f}".format(x), mm))
        row.extend(fmm)
        vals.append(row)
        

        
cols=['lr','b_sz','val_mse','val_mae','val_r2','val_pcc','val_spc']
fig, ax = plt.subplots(1, figsize=(20,20))
table   = ax.table(vals,
                cellLoc="center",
                colLabels=cols,
                rowLabels=list(range(0,len(vals))))

table.set_fontsize(14)
ax.axis('off')
plt.show()

#to csv
# TODO problem, right now the results are split by regression types
import pandas as pd
ar_val   = np.array(vals)
ar_col   = np.expand_dims(np.array(cols),0)
csv_data = np.vstack((ar_col, ar_val))
pd.DataFrame(csv_data).to_csv("Experiments/Table_6_Linear_Regression_Tunning/Sup_6_Results.csv")

In [ ]:
dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type="insulation",
                  label_val=10)
dm.setup()
path="/home/heracles/Documents/Professional/Research/Honeydew/Experiments/Table_4_Linear_Regression_Tunning/insulation_10_RFalse_LFalse/optuna/version_5/checkpoints/epoch=49-step=197949.ckpt"
model = lrg.LinearRegressionModule.load_from_checkpoint(path).cuda()
print(ev.getModelMetrics(model, dm, 'train'))
ev.createPlot(model, dm, "train","l.png")

In [ ]:
#to csv
# TODO problem, right now the results are split by regression types
import pandas as pd
ar_val   = np.array(vals)
ar_col   = np.expand_dims(np.array(cols),0)
csv_data = np.vstack((ar_col, ar_val))
pd.DataFrame(csv_data).to_csv("Experiments/Table_4_Linear_Regression_Tunning/Sup_4_Results.csv")